In [1]:
import cobra
import pandas as pd
from collections import OrderedDict
from copy import deepcopy
import math

In [2]:
def build_reaction_equation_from_metabolites_dict(met_dict, arrow='<=>', floatdecimal=6):
    lhs = []; rhs = [];
    for k,v in met_dict.items():
        v = float(v)
        if v == -1:
            lhs.append(k)
        elif v == 1:
            rhs.append(k)
        elif v < 0 and v != -1 and v.is_integer():
            lhs.append(' '.join([str(-int(v)), k]))
        elif v > 0 and v != 1 and v.is_integer():
            rhs.append(' '.join([str(int(v)), k]))
        elif v < 0 and v != -1:
            lhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(-v), k]))
        elif v > 0 and v != 1:
            rhs.append(' '.join([('{:.' + str(floatdecimal) + 'f}').format(v), k]))
    return ' '.join([ ' + '.join(lhs), arrow, ' + '.join(rhs)])

In [3]:
modelpath = '../../build_model/input/iRhtoC-old.json'
# modelpath = '../../build_model/input/iRhtoN.json'
# modelpath = '../../build_model/input/iRhtoBatch-Rabinowitz.json'
old_lipid_frac = 0.28981020686 # lipid mass fraction in biomass equation; update only if changing the lipid fraction based on data
# model = deepcopy(cobra.io.load_json_model('../../build_model/input/release1-iRhtoC.json'))
# model = deepcopy(cobra.io.load_json_model('../../build_model/input/release2-iRhtoC.json'))
model = deepcopy(cobra.io.load_json_model(modelpath))
# select biomass rxns; by default, select rxns with "sbo" as SBO:0000629
biomass_rxns = []
for rxn in model.reactions:
    if 'biomass' in rxn.id.lower():
        biomass_rxns.append(rxn.id)
        break
    elif 'sbo' in rxn.annotation:
        if 'SBO:0000629' in rxn.annotation['sbo']:
            biomass_rxns.append(rxn.id)
            break
print('biomass_rxns:',biomass_rxns)
ngam_0 = model.reactions.get_by_id('ATPM_c').lower_bound

model.reactions.EX_glc__D_e.bounds = (0,1000)
model.reactions.EX_o2_e.bounds = (-1000,1000)
model.reactions.ATPM_c.bounds = (0,1000)
# remove cobalt, cu, and btn from medium,
# to be consistent with the medium from https://www.sciencedirect.com/science/article/pii/S0168165613003490?via%3Dihub
model.reactions.EX_cobalt2_e.bounds = (0,0)
# model.reactions.EX_cu2_e.bounds = (0,0)
# model.reactions.EX_btn_c.bounds = (0,0)
# compare medium to paper
medium = {'EX_nh4_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_o2_e': 1000, 'EX_pi_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_h_e': 1000.0, 'EX_k_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_zn2_e': 1000.0, 'EX_btn_c': 0.01}

if model.medium != medium:
    for exchange_reaction, bounds in medium.items():
        if exchange_reaction in model.reactions:
            reaction = model.reactions.get_by_id(exchange_reaction)
            if model.medium[exchange_reaction] != bounds:
                print(f"{exchange_reaction} bounds do not match: {reaction.bounds}")
        else:
            print(f"{exchange_reaction} not found in the model.")

print('medium in model:'+str(model.medium))
# from SC version
# model.reactions.HCO3E_e.bounds = (0,0)
# Added on 2022-May-06
# model.reactions.FDH_c.knock_out()
# model.reactions.ABTA_c.knock_out()
# model.reactions.get_by_id('4ABTORy_c').knock_out()
# model.reactions.get_by_id('4ABTORx_c').knock_out()
# model.reactions.FADH2t_c_m.knock_out()

# Add NADH oxidase reaction
# rxn = cobra.Reaction('NADHOX_c')
# model.add_reactions([rxn])
# rxn.reaction = 'nadh_c + o2_c + h_c --> nad_c + h2o2_c'
# model.reactions.NADHOX_c.bounds = (0,0)

model.objective = dict() # clears the objective
model.reactions.ATPM_c.objective_coefficient = 1
default_bounds = {rxn.id:rxn.bounds for rxn in model.reactions}
print(model.reactions.ATPM_c.bounds)

biomass_rxns: ['BIOMASS']
medium in model:{'EX_4abz_e': 0.25, 'EX_btn_c': 0.01, 'EX_ca2_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_cu2_e': 1000.0, 'EX_fe2_e': 1000.0, 'EX_fol_e': 0.25, 'EX_h2o_e': 1000.0, 'EX_h_e': 1000.0, 'EX_inost_e': 0.25, 'EX_k_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_nac_e': 0.25, 'EX_nh4_e': 1000.0, 'EX_ni2_e': 1000.0, 'EX_o2_e': 1000, 'EX_pi_e': 1000.0, 'EX_pnto__R_e': 0.25, 'EX_pydxn_e': 0.25, 'EX_ribflv_e': 0.25, 'EX_so4_e': 1000.0, 'EX_thm_e': 0.25, 'EX_zn2_e': 1000.0}
(0, 1000)


# Processing data and running model

In [4]:
df_data = pd.read_excel('./ATPM_data.xlsx', sheet_name='Data')
df_data = df_data.loc[:]
# df_data.index = df_data.Dataset.to_list()
# df_data.index = df_data.apply(lambda row: [row['Dataset'], row['Growth_rate']], axis=1).tolist()

# filter out data points
df_data = df_data[df_data.Dataset == 'Shen2013_chemoClimAero']
# df_data = df_data[df_data.Dataset == 'Shen2013_chemoNlimAero']
# df_data = df_data[df_data.Dataset == 'Rabinowitz2024_batchAero']

df_data

,Dataset,Year,Group,pmid,Simulation_settings,Growth_rate,Oxy,Substrate_uptake,Oxy_uptake,Secretions,...,Note,Culture,Substrate conc (g/L),Rpm,Aero_vvm,Working volume (L),Controlled_O2_saturation (%),Column3,Single_point_GAM,Strain
0,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:0.185172159189104,0.02,NaN,EX_glc__D_e:U:0.289324541639202,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-60.000000,AS 2.1389
1,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:0.38874606514305,0.04,NaN,EX_glc__D_e:U:0.574214990686233,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-30.000000,AS 2.1389
2,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:0.605128906492802,0.06,NaN,EX_glc__D_e:U:0.856400643252039,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-20.000000,AS 2.1389
3,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:0.775407735831279,0.08,NaN,EX_glc__D_e:U:1.08601530847179,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-15.000000,AS 2.1389
4,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:1.34142068640932,0.13,NaN,EX_glc__D_e:U:1.80399209573925,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-9.230769,AS 2.1389
5,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:1.51965140698119,0.15,NaN,EX_glc__D_e:U:2.04368335321509,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-8.000000,AS 2.1389
6,Shen2013_chemoClimAero,2013,Chemo,23965273.0,EX_nh4_e:U:1.9848454953271,0.19,NaN,EX_glc__D_e:U:2.58038213287047,NaN,NaN,...,NaN,Chemo,NaN,500.0,0.8,1.85,NaN,NaN,-6.315789,AS 2.1389


In [5]:
# Remove GAM from biomass equation

for biom in biomass_rxns:
    print('old biomass equation:')
    display(model.reactions.get_by_id(biom).reaction)
    adp = model.metabolites.adp_c
    gam0 = model.reactions.get_by_id(biom).metabolites[adp]
    met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
    for met in ['atp_c', 'h2o_c']:
        met_dict[met] += gam0
    for met in ['adp_c', 'pi_c', 'h_c']:
        met_dict[met] -= gam0
    biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
    model.reactions.get_by_id(biom).reaction = biom_eqn
print('new biomass equation:')
display(model.reactions.get_by_id(biom).reaction)
# show equation in JSON form
biomass_dict = {met.id: model.reactions.get_by_id(biom).metabolites[met] for met in model.reactions.get_by_id(biom).metabolites}
print('metabolites:',str(biomass_dict).replace("'",'"'))

old biomass equation:


'1.390551 13BDglucan_c + 0.365935 16BDglucan_en + 0.021534 G00006_c + 0.316243 alatrna_c + 0.0001 amet_c + 0.124943 argtrna_c + 0.150191 asntrna_c + 0.150191 asptrna_c + 213.618525 atp_c + 0.0001 btn_c + 0.001284 ca2_c + 0.032876 chtn_c + 0.0001 coa_c + 0.05844 ctp_c + 0.000114 cu2_c + 0.004532 cystrna_c + 0.004874 datp_c + 0.007607 dctp_c + 0.007607 dgtp_c + 2.9e-05 docosa_c + 0.004874 dttp_c + 0.072767 epist_c + 0.0001 fad_c + 0.000663 fe2_c + 0.250534 glntrna_c + 0.250534 glutrna_c + 0.287758 glytrna_c + 0.023598 gpianchorSC_r + 0.051206 gtp_c + 213.585046 h2o_c + 0.000339 hdca_c + 0.0001 hemeA_c + 0.062472 histrna_c + 0.190652 iletrna_c + 0.005733 ipc_g + 0.602526 k_c + 0.259274 leutrna_c + 0.000865 linoea_c + 0.000232 linolen_c + 0.212663 lystrna_c + 0.036901 mettrna_c + 0.063489 mg2_c + 9.4e-05 mn2_c + 0.0001 nad_c + 0.0001 nadp_c + 0.000226 ocdca_c + 0.000414 ocdcea_c + 0.066377 oglycanSC_g + 0.015562 pail_c + 0.029204 pc_c + 0.007054 pe_c + 0.121707 phetrna_c + 0.136597 protrna

new biomass equation:


'1.390551 13BDglucan_c + 0.365935 16BDglucan_en + 0.021534 G00006_c + 0.316243 alatrna_c + 0.0001 amet_c + 0.124943 argtrna_c + 0.150191 asntrna_c + 0.150191 asptrna_c + 0.033479 atp_c + 0.0001 btn_c + 0.001284 ca2_c + 0.032876 chtn_c + 0.0001 coa_c + 0.05844 ctp_c + 0.000114 cu2_c + 0.004532 cystrna_c + 0.004874 datp_c + 0.007607 dctp_c + 0.007607 dgtp_c + 2.9e-05 docosa_c + 0.004874 dttp_c + 0.072767 epist_c + 0.0001 fad_c + 0.000663 fe2_c + 0.250534 glntrna_c + 0.250534 glutrna_c + 0.287758 glytrna_c + 0.023598 gpianchorSC_r + 0.051206 gtp_c + 0.000339 hdca_c + 0.0001 hemeA_c + 0.062472 histrna_c + 0.190652 iletrna_c + 0.005733 ipc_g + 0.602526 k_c + 0.259274 leutrna_c + 0.000865 linoea_c + 0.000232 linolen_c + 0.212663 lystrna_c + 0.036901 mettrna_c + 0.063489 mg2_c + 9.4e-05 mn2_c + 0.0001 nad_c + 0.0001 nadp_c + 0.000226 ocdca_c + 0.000414 ocdcea_c + 0.066377 oglycanSC_g + 0.015562 pail_c + 0.029204 pc_c + 0.007054 pe_c + 0.121707 phetrna_c + 0.105458 pi_c + 0.136597 protrna_c + 

metabolites: {"13BDglucan_c": -1.390551, "16BDglucan_en": -0.365935, "G00006_c": -0.021534, "alatrna_c": -0.316243, "amet_c": -0.0001, "argtrna_c": -0.124943, "asntrna_c": -0.150191, "asptrna_c": -0.150191, "atp_c": -0.033479, "btn_c": -0.0001, "ca2_c": -0.001284, "chtn_c": -0.032876, "coa_c": -0.0001, "ctp_c": -0.05844, "cu2_c": -0.000114, "cystrna_c": -0.004532, "datp_c": -0.004874, "dctp_c": -0.007607, "dgtp_c": -0.007607, "docosa_c": -2.9e-05, "dttp_c": -0.004874, "epist_c": -0.072767, "fad_c": -0.0001, "fe2_c": -0.000663, "glntrna_c": -0.250534, "glutrna_c": -0.250534, "glytrna_c": -0.287758, "gpianchorSC_r": -0.023598, "gtp_c": -0.051206, "hdca_c": -0.000339, "hemeA_c": -0.0001, "histrna_c": -0.062472, "iletrna_c": -0.190652, "ipc_g": -0.005733, "k_c": -0.602526, "leutrna_c": -0.259274, "linoea_c": -0.000865, "linolen_c": -0.000232, "lystrna_c": -0.212663, "mettrna_c": -0.036901, "mg2_c": -0.063489, "mn2_c": -9.4e-05, "nad_c": -0.0001, "nadp_c": -0.0001, "ocdca_c": -0.000226, "oc

In [6]:
# all lipids in biomass, in case lipid composition is updated
bio_lipids = ['epist_c','tag_c','hdca_c','ocdca_c','ocdcea_c','linoea_c','linolen_c','docosa_c','ttcosa_c','pail_c','pc_c','pe_c','ps_c','ipc_g']
# calculate lipid fraction in old biomass equation
# old_lipid_frac = 0
# for lipid in bio_lipids:
# 	old_lipid_frac += model.metabolites.get_by_id(lipid).formula_weight	* model.reactions.get_by_id(biomass_rxns[0]).metabolites[model.metabolites.get_by_id(lipid)]
# old_lipid_frac /= -1000
# print('calculated old_lipid_frac:',old_lipid_frac)

In [7]:
data={}
biomass_equations = {}
print('\t'.join(['Status','Substrate uptake rate', 'Growth rate', 'ATPM', 'Extra info (if any)']))

for i in df_data.index:
    with model as m:
        extra_info = []
        if df_data.Remove_tag[i] == 'Removed':
            print('', '')
            continue
        rxns_tuned = []
        
        # Implement oxygen conditions. If anaerobic, supplement nutrient
        if df_data.Oxy[i] == 0:
            m.reactions.EX_o2_e.bounds = (0,1000)
            rxns_supp = ['EX_ergst_e', 'EX_hdcea_e', 'EX_ocdcea_e', 'EX_nac_e', 'EX_pnto__R_e']
            rxns_tuned += rxns_supp
            for r in rxns_supp:
                m.reactions.get_by_id(r).bounds = (-1000,1000)
        else:
            m.reactions.EX_o2_e.bounds = (-1000,1000)
                
        # Select biomass reaction and implement growth rate
        gr = df_data.Growth_rate[i]
        rxns_tuned += biomass_rxns
        m.reactions.get_by_id(biomass_rxns[0]).bounds = (gr,gr)
        
        # Implement substrate uptake
        sub_upt = str(df_data.Substrate_uptake[i]).split(' | ')
        for s in sub_upt:
            r,ctype,v = str(s).split(':')  # Convert Series object to string
            rxns_tuned.append(r)
            v = float(v)
            substrate = r
            if ctype == 'E':
                m.reactions.get_by_id(r).bounds = (-v,-v)
            elif ctype == 'U':
                m.reactions.get_by_id(r).bounds = (-v,1000)
            else:
                print('There is a typo in ' + s)
                
        # Implement oxygen uptake
        o2_upt = df_data.Oxy_uptake[i]
        if pd.isnull(o2_upt) == False:
            r,ctype,v = str(o2_upt).split(':')  # Convert Series object to string
            v = float(v)
            if ctype == 'E':
                m.reactions.get_by_id(r).bounds = (-v,-v)
            elif ctype == 'U':
                m.reactions.get_by_id(r).bounds = (-v,1000)
            else:
                print('There is a typo in ' + s)
                
        # Implement secretions
        secs = df_data.Secretions[i]
        if pd.isnull(secs) == False:
            secs = str(secs).split(' | ')  # Convert Series object to string
            for s in secs:
                r,ctype,v = s.split(':')
                rxns_tuned.append(r)
                v = float(v)
                if ctype == 'E':
                    m.reactions.get_by_id(r).bounds = (v,v)
                elif ctype == 'L':
                    m.reactions.get_by_id(r).bounds = (v,1000)
                else:
                    print('There is a typo in ' + s)
                
        # Implement notes
        if pd.isnull(df_data.Simulation_settings[i]) == False:
            s = df_data.Simulation_settings[i]
            # each setting separated by ','
            settings = s.split(',')
            for s in settings:
                try:
                    if s == 'Add_NADH_oxidase':
                        rxns_tuned.append('NADHOX_c')
                        m.reactions.NADHOX_c.bounds = (0,1000)
                    elif s == 'Disable_NH4_uptake':
                        rxns_tuned.append('EX_nh4_e')
                        m.reactions.EX_nh4_e.bounds = (0,1000)
                    elif 'new_lipid_frac' in s:
                        new_lipid_frac = float(s.split('=')[-1])
                        # round down, reducing the last decimal by 1
                        # decimals = str(new_lipid_frac).split('.')[-1]
                        # if len(decimals) > 1:
                        #     new_lipid_frac = math.ceil(new_lipid_frac*10**len(decimals)+0.1)/10**len(decimals)
                            # new_lipid_frac = math.floor(new_lipid_frac*10**len(decimals)+0.1)/10**len(decimals)
                        met_dict = OrderedDict({met.id:v*(1-new_lipid_frac)/(1-old_lipid_frac) for met,v in m.reactions.get_by_id(biomass_rxns[0]).metabolites.items()})
                        for met in bio_lipids:
                            met_dict[met] = met_dict[met] * new_lipid_frac / (old_lipid_frac * ((1-new_lipid_frac)/(1-old_lipid_frac)))
                        biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
                        m.reactions.get_by_id(biomass_rxns[0]).reaction = biom_eqn
                        m.reactions.get_by_id(biomass_rxns[0]).bounds = (gr,gr)
                        # output biomass equation
                        extra_info.append('new lipid mass fraction: ' + str(new_lipid_frac))
                    else:
                        r,ctype,v = s.split(':')
                        v = float(v)
                        if ctype == 'E':
                            m.reactions.get_by_id(r).bounds = (v,v)
                        elif ctype == 'L':
                            m.reactions.get_by_id(r).lower_bound = v
                        elif ctype == 'U':
                            m.reactions.get_by_id(r).upper_bound = v
                        else:
                            print('There is a typo in ' + s)
                except Exception as e:
                    print(e)
                    print('Setting does not fit usual format: ' + s)
                
        # Implement mean value fix assuming experimental error are roughly 10% of mean value
        adjs = df_data.Mean_value_adjustment[i]
        if pd.isnull(adjs) == False:
            adjs = adjs.split(' | ')
            for adj in adjs:
                r,var,val = adj.split(':')
                val = float(val)
                if var == 'LB':
                    m.reactions.get_by_id(r).lower_bound = val
                elif var == 'UB':
                    m.reactions.get_by_id(r).upper_bound = val
                else:
                    print('There is a typo in ' + adj)
        
        # FBA maximizing ATPM
        fba = m.optimize()
        # if infeasible, find the highest possible growth rate w/o enforcing the measured bounds
        if fba.status != 'optimal':
            m.reactions.get_by_id(biomass_rxns[0]).bounds = (0,1000)
            maxgrowth = m.optimize()
            # print the highest possible growth rate
            print('Optimal solution not found. Max growth rate:',maxgrowth[biomass_rxns[0]])
        # m.summary()
        gr = fba[biomass_rxns[0]]
        extra_info.append('NH4 uptake: ' + str(fba['EX_nh4_e']))
        output = [fba.status, fba[substrate], gr, round(fba['ATPM_c'],4), ', '.join(extra_info)]
        print('\t'.join([str(item) for item in output]))
        data[gr] = fba['ATPM_c']
        # for each growth rate, record the biomass components and their coefficients
        met_dict = OrderedDict({met.id:v for met,v in m.reactions.get_by_id(biomass_rxns[0]).metabolites.items()})
        biomass_equations[gr] = met_dict
        
        for r in rxns_tuned:
            m.reactions.get_by_id(r).bounds = default_bounds[r]
# make a table with the biomass coefficients for each growth rate
df_biomass = pd.DataFrame(biomass_equations).T
df_biomass.to_csv('maxATPM_biomass_equations.csv')

Status	Substrate uptake rate	Growth rate	ATPM	Extra info (if any)
optimal	-0.289324541639202	0.02	5.1277	NH4 uptake: -0.1067910599999995
optimal	-0.574214990686233	0.04	10.0753	NH4 uptake: -0.2135821199999991
optimal	-0.856400643252039	0.06	14.9131	NH4 uptake: -0.3203731799999967
optimal	-1.08601530847179	0.08	17.6165	NH4 uptake: -0.42716423999999753
optimal	-1.80399209573925	0.13	30.2191	NH4 uptake: -0.6941418899999905
optimal	-2.04368335321509	0.15	33.3316	NH4 uptake: -0.8009329499999909
optimal	-2.58038213287047	0.19	41.8838	NH4 uptake: -1.0145150699999936


In [8]:
model.medium

{'EX_4abz_e': 0.25,
 'EX_btn_c': 0.01,
 'EX_ca2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_fol_e': 0.25,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_inost_e': 0.25,
 'EX_k_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_nac_e': 0.25,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_o2_e': 1000,
 'EX_pi_e': 1000.0,
 'EX_pnto__R_e': 0.25,
 'EX_pydxn_e': 0.25,
 'EX_ribflv_e': 0.25,
 'EX_so4_e': 1000.0,
 'EX_thm_e': 0.25,
 'EX_zn2_e': 1000.0}

In [9]:
# find slope of the data
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
# if there's only one data point, add one at growth rate 0 and ATPM as the model's NGAM
if len(data) == 1:
	print('Only one data point found. Adding one at growth rate 0 and ATPM_c lower bound as NGAM.')
	if 0 not in data:
		data[0] = ngam_0
print(data)
x = np.array(list(data.keys()))
y = np.array(list(data.values()))
slope, intercept, r_value, p_value, std_err = linregress(x, y)
# calculate GAM and NGAM
# if intercept is less than 0, print warning
gam = slope
ngam = intercept
print(slope, intercept, r_value, p_value, std_err)
if intercept < 0:
	# update values below once recalculated
	ngam = 1.189
	gam = 183.6717
	# show error message
	raise ValueError('Intercept is negative. Recalculate with Excel or another method.')

# x = x[:,np.newaxis]
# a, _, _, _ = np.linalg.lstsq(x, y)
# print(a)

# plt.plot(x, y, 'bo')
# plt.plot(x, a*x, 'r-')
# plt.show()

{0.02: 5.127655622180527, 0.04: 10.075287085118905, 0.06: 14.913103810919576, 0.08: 17.616538450470372, 0.13: 30.219094092601605, 0.15: 33.33163837774966, 0.19: 41.883767295015865}
215.4570718535822 1.258692370308065 0.9988059328382732 9.456879639628307e-08 4.712966254358325


In [10]:
# update ATPM in model to add NGAM/GAM
for biom in biomass_rxns:
    adp = model.metabolites.adp_c
    met_dict = OrderedDict({met.id:v for met,v in model.reactions.get_by_id(biom).metabolites.items()})
    for met in ['atp_c', 'h2o_c']:
        if met in met_dict:
            met_dict[met] -= gam
        else:
            met_dict[met] = -gam
    for met in ['adp_c', 'pi_c', 'h_c']:
        if met in met_dict:
            met_dict[met] += gam
        else:
            met_dict[met] = gam
    biom_eqn = build_reaction_equation_from_metabolites_dict(met_dict, arrow='-->')
    model.reactions.get_by_id(biom).reaction = biom_eqn
display(model.reactions.get_by_id(biom).reaction)
model.reactions.ATPM_c.bounds = (ngam,1000)

'1.390551 13BDglucan_c + 0.365935 16BDglucan_en + 0.021534 G00006_c + 0.316243 alatrna_c + 0.0001 amet_c + 0.124943 argtrna_c + 0.150191 asntrna_c + 0.150191 asptrna_c + 215.490551 atp_c + 0.0001 btn_c + 0.001284 ca2_c + 0.032876 chtn_c + 0.0001 coa_c + 0.05844 ctp_c + 0.000114 cu2_c + 0.004532 cystrna_c + 0.004874 datp_c + 0.007607 dctp_c + 0.007607 dgtp_c + 2.9e-05 docosa_c + 0.004874 dttp_c + 0.072767 epist_c + 0.0001 fad_c + 0.000663 fe2_c + 0.250534 glntrna_c + 0.250534 glutrna_c + 0.287758 glytrna_c + 0.023598 gpianchorSC_r + 0.051206 gtp_c + 215.457072 h2o_c + 0.000339 hdca_c + 0.0001 hemeA_c + 0.062472 histrna_c + 0.190652 iletrna_c + 0.005733 ipc_g + 0.602526 k_c + 0.259274 leutrna_c + 0.000865 linoea_c + 0.000232 linolen_c + 0.212663 lystrna_c + 0.036901 mettrna_c + 0.063489 mg2_c + 9.4e-05 mn2_c + 0.0001 nad_c + 0.0001 nadp_c + 0.000226 ocdca_c + 0.000414 ocdcea_c + 0.066377 oglycanSC_g + 0.015562 pail_c + 0.029204 pc_c + 0.007054 pe_c + 0.121707 phetrna_c + 0.136597 protrna

In [11]:
# write new version of model to file
# Save the model to the specified file path

# modelpath_new = modelpath.split('.json')[0] + '_updated_ATPM.json'
modelpath_new = modelpath
cobra.io.save_json_model(model, modelpath_new)